### Description

This notebook provides an overview of the saccade extraction workflow.

### Import all the necesarry functions

In [1]:
from saccade_extraction import (
    initializeProject,
    addNewSessions,
    labelPutativeSaccades,
    createTrainingDataset,
    trainModels,
    extractRealSaccades,
    validatePredictions
)

### 1. Initialize a project

The `initializeProject` function will make a new directory with a standard file structure and generate a config file. You will only need to do this once. The config file specifies the following parameters:
* `velocityThreshold` - Velocity threshold for peak detection (in percentiles)
* `minimumPeakDistance` - Minimum distance between putative saccades (in seconds) for peak detection
* `responseWindow` - Time window around saccades
* `nFeatures` - Number of time points for resampling

In [2]:
projectDirectory = '/home/josh/Desktop/dummy_project'
configFile = initializeProject(projectDirectory)
print(f'You can find the config file here: {configFile}')

You can find the config file here: /home/josh/Desktop/dummy_project/config.yaml


### 2. Extract putative saccades for labeling ###

The `addNewSessions` function processes the pose estimates and extracts high-velocity eye movements, i.e., putative saccades. You need to pass it a list of tuples in which each tuple contains the file path to the pose estimate output by DeepLabCut and the filepath to the file that stores the inter-frame intervals for that video recording. This only needs to be done if you intend to collect training data from a particular recording.

In [ ]:
fileSets = [(
    '/home/jbhunt/Dropbox/Data/20240402_unitME_session002_rightCam-0000DLC_resnet50_sacnetJan29shuffle1_1030000.csv',
    '/home/jbhunt/Dropbox/Data/20240402_unitME_session002_rightCam_timestamps.txt',
)]
addNewSessions(configFile, fileSets)

### 3. Label a subset of the putative saccades

Use the saccade labeling GUI to collect information about the putative saccades. For each putative saccade you wish to label, follow this workflow:
1. Open any of the folders you created by calling the `addNewSessions` function by clicking on the `Open` button.
2. Use the radio buttons on the left to indicate the direction of the saccade. Use the description on the y-axis of the top-left plot to judge whether the saccade moves the eye in the nasal or temporal direction. If you don't think the putative saccade is a real saccade, select the "Not a saccade" radio button.
3. Indicate when the saccade starts. Make sure you have enabled the `Start` line selector checkbox, then click anywhere on the Matplotlib figure. You should see the left vertical line turn green
4. Indicate when the saccade stops. Make sure you have enabled the `Stop` line selector checkbox, then click anywhere on the Matplotlib figure. You should see the right vertical line turn red.
5. Save your progress by clicking the `Save` button.

gui = labelPutativeSaccades(configFile)

### 4. Create a training dataset

The `createTrainingDataset` function will collect all manually collected information and generate a dataset that will be used for training below. You only need to run this funciton if you have collected new training data since you last trained the models.

In [ ]:
createTrainingDataset(configFile)

### 5. Train your models

The `trainModels` function will use cross-validation to search a hyperparameter space for the unique combination of hyperparameters that result in the best classification and regression performance. This function accepts the keyword argument `trainingDatasetIndex` which specifies which training dataset to train on. The default behavior is for the models to train on the most recently generated training dataset, i.e., `trainingDatasetIndex=-1`. You only need to run this function if you have generated a new training dataset.

In [ ]:
trainModels(configFile, trainingDatasetIndex=-1)

### 6. Extract real saccades

The `extractRealSaccades` function uses the models trained above to extract real saccades from the DeepLabCut pose estimates. It works much like the `addNewSessions` function in that you need to pass it a list of tuples where each tuple contains the path to a pose estimate and the path to the associated inter-frame intervals file. You can process as many recordings at once as you want. Currently, this functions saves these 2 files in the same directory as the pose estimates:
* `real_saccade_waveforms.npy` (N saccades x M features) - The positional saccade waveforms for all real saccades
* `real_saccade_labels.npy` (N saccades x 3) - The predicted saccade direction (column 1) and the fractional frame index for the saccade onset (column 2) and offset (column 3)

In [ ]:
fileSets = [
    ('Path to pose estimates', 'Path to inter-frame intervals file'),
    ('Another path to pose estimates', 'Another path to inter-frame intervals file')
]
extractRealSaccades(
    configData,
    fileSets,
)

### 7. Validation

To validate the predictions made by the models, you can use the `validatePredictions` function. This will plot the nasal-temporal component of the eye position, then plot the start of each saccade as a vertical line. The direction of the saccade is indicated by the color of the vertical lines.

In [ ]:
dlcFile = 'Path to pose estimates'
saccadePredictionsFile = 'Path to real_saccade_labels.npy file'
fig, ax = validatePredictions(
    dlcFile,
    saccadePredictionsFile,
)